In [87]:
from datetime import datetime
import numpy as np

In [88]:
class Discount:
    def __init__(self, value, description=None):
        if value < 0:
            raise ValueError("Discount value cannot be negative.")
        self.value = value
        self.description = description

    def apply_discount(self, price):
        return price * (1 - self.value / 100)

    def __str__(self):
        return f"{self.value}% off"

class Product:
    def __init__(self,name,category,stock,price,code):
        if stock < 0:
            raise ValueError("Stock cannot be negative.")
        if price < 0:
            raise ValueError("Price cannot be negative.")
        self.name=name
        self.category=category
        self.stock=stock
        self.price=price
        self.code=code

    def update_price(self,increasement):
        self.price=self.price * (1+(increasement/100))

    def update_stock(self,sold_stock):
        self.stock=self.stock-sold_stock

    def __str__(self):
        return f"{self.name}({self.category}): stock={self.stock}, price={self.price:.2f}, id={self.code}"
        
class PerishableProduct(Product):
    def __init__(self,name,category,stock,price,code,expiration_date):
        super().__init__(name, category, stock, price, code)
        self.expiration_date=datetime.strptime(expiration_date,"%Y-%m-%d")
    def is_expired(self):
        return datetime.now()>self.expiration_date
    def __str__(self):
        status = "Expired" if self.is_expired() else "Valid"
        return super().__str__() + f", Expiration Date: {self.expiration_date.date()} ({status})"

class ElectronicProduct(Product):
    def __init__(self,name,category,stock,price,code,ram,battery,cpu):
        super().__init__(name, category, stock, price, code)
        self.ram=ram
        self.battery=battery
        self.cpu=cpu
    def __str__(self):
        return super().__str__() + f", RAM:{self.ram}, Battery:{self.battery}, cpu:{self.cpu}"

class Inventory:
    def __init__(self):
        self.products={}

    def add_product(self,product):
        if product.code in self.products:
            raise ValueError("This product is already in the inventory.")
        self.products[product.code]=product

    def remove_product(self,product):
        if product.code not in self.products:
            raise ValueError("This product is not in the inventory")
        del self.products[product.code]
    def update_product_price(self,code,price_increase):
        self.products[code].update_price(price_increase)

    def update_product_stock(self,code,items_sold):
        self.products[code].update_stock(items_sold)

    def get_all_products(self):
        return list(self.products.values())

    def __str__(self):
        return f"Inventory. Total number of items:{len(self.products)}"
    
class OrdersHistory:
    def __init__(self):
        self.orders=[]

    def add_order(self,order):
        self.orders.append(order)

    def get_all_orders(self):
        return self.orders
    
    def __str__(self):
        return f"Order History: {len(self.orders)} orders"

class Store:
    def __init__(self,name,location,code):
        self.name = name
        self.location = location
        self.code = code
        self.inventory=Inventory()
        self.orders_history=OrdersHistory()
    
    def list_all_products(self):
        products=self.inventory.get_all_products()
        if not products:
            print(f"The store {self.name}has no inventory.")
        else:
            print(f"Products in the store {self.name}:")
            for product in products:
                print(product)
    def list_all_orders(self):
        orders=self.orders_history.get_all_orders()
        if not orders:
            print(f"The store {self.name}has no orders.")
        else:
            print(f"Orders in the store {self.name}:")
            for order in orders:
                print(order)
    def update_location(self,new_location):
        self.location=new_location

    def __str__(self):
        return f"{self.name} located in {self.location} with an id {self.code}"

class StoreOnline(Store):
    def __init__(self,name,location,code,url):
        super().__init__(name,location,code)
        self.url=url

    def __str__(self):
        return super().__str__() + f", Online URL:{self.url}"

class Customer:
    def __init__(self,name,address,electronic_email):
        self.name = name
        self.address = address
        self.electronic_email = electronic_email
    def __str__(self):
        return f"Name:{self.name}, Address:{self.address},email:{self.electronic_email}"

class Orders:
    def __init__(self,order_id,tienda,products_quantities_dicc,customer,distance,discount):
        self.order_id = order_id
        self.tienda=tienda
        self.products_quantities_dicc=products_quantities_dicc
        self.customer = customer
        self.status="Pending"
        self.distance=distance
        self.discount=discount
    def cost_per_transport(self):
        cost_base = 2
        price_per_km = 0.1
        cost = cost_base if self.distance <= 10 else cost_base + price_per_km * (self.distance - 10)    
        return cost
    
    def total_cost_calculation(self):
        total_cost=0
        for p,q in self.products_quantities_dicc.items():

            if q > p.stock:
                print(f"The maximum amount of products available is {p.stock}")
                q=p.stock
            total_cost += q*p.price
            if self.discount:
                total_cost -= p.price * (1-(self.discount.value/100))
        total_cost += self.cost_per_transport()
        return total_cost 

    def set_status(self, new_status):
        self.status=new_status

    def process_order(self):
        products_to_send=0
        for p,q in self.products_quantities_dicc.items():
            if q > p.stock:
                products_to_send=p.stock
                self.tienda.inventory.update_product_stock(p.code,products_to_send)
            else:
                products_to_send=q
                self.tienda.inventory.update_product_stock(p.code,products_to_send)
        self.set_status('Processed')
        
    def __str__(self):
        return f"Costumer {self.customer}, Tienda {self.tienda.code}, Identificador {self.order_id},Total: {self.total_cost_calculation():.2f}, Status: {self.status}"

class Transport:
    def __init__(self,order,driver_name):
        self.order=order
        self.driver_name=driver_name
    def finish_order(self):
        self.order.set_status("Delivered")
    
class StoreManager:
    def __init__(self):
        self.stores={}
    def add_store(self,store):
        if store.name in self.stores:
            raise ValueError("This store is already in the storemaneger.")
        self.stores[store.code]=store
    def remove_store(self,store):
        if store.code not in self.stores:
            raise ValueError("This store is not in the storemaneger")
        del self.stores[store.code]
    
    def update_store_location(self,code,new_location):
        self.stores[code].update_location(new_location)

    def list_all_stores(self):
        for i in self.stores.values():
            print(i)

    def __str__(self):
        return f"Store Manager. Total number of stores:{len(self.stores)} "

In [89]:
cost=-np.inf

In [90]:
cost =0

In [91]:
product1=PerishableProduct('Leche de Avena','Lacteo',10,1.50,'P001',"2025-05-15")

In [92]:
product2=PerishableProduct('Leche de coco','Lacteo',10,1.70,'P002',"2024-05-15")

In [93]:
product3=PerishableProduct('Platano','Fruta',1000,2.10,'P003',"2025-05-15")

In [94]:
product4=ElectronicProduct('Dell I9','Ordenador',200,2000,'PE004',8,5000,10)

In [95]:
tienda1=StoreOnline('Amazon','Luxembourg city,Luxembourg','TO0001','www.amazon.lu')

In [96]:
tienda1.inventory.add_product(product1)
tienda1.inventory.add_product(product2)
tienda1.inventory.add_product(product3)
tienda1.inventory.add_product(product4)


In [97]:
tienda1.list_all_products()

Products in the store Amazon:
Leche de Avena(Lacteo): stock=10, price=1.50, id=P001, Expiration Date: 2025-05-15 (Valid)
Leche de coco(Lacteo): stock=10, price=1.70, id=P002, Expiration Date: 2024-05-15 (Expired)
Platano(Fruta): stock=1000, price=2.10, id=P003, Expiration Date: 2025-05-15 (Valid)
Dell I9(Ordenador): stock=200, price=2000.00, id=PE004, RAM:8, Battery:5000, cpu:10


In [98]:
lista_de_tiendas=StoreManager()

In [99]:
lista_de_tiendas.add_store(tienda1)

In [100]:
lista_de_tiendas.list_all_stores()

Amazon located in Luxembourg city,Luxembourg with an id TO0001, Online URL:www.amazon.lu


In [101]:
customer1=Customer('Gabi','Mislata,Valencia','gabi@vlc.lux')

In [102]:
discount1=Discount(5,'Black Friday')

In [103]:
order1=Orders('order001',tienda1,{product4:2},customer1,100,discount1)

In [104]:
transport1=Transport(order1,'Juan Vazquez')

In [105]:
order1.process_order()

In [106]:
print(order1)

Costumer Name:Gabi, Address:Mislata,Valencia,email:gabi@vlc.lux, Tienda TO0001, Identificador order001,Total: 2111.00, Status: Processed
